<a href="https://colab.research.google.com/github/Abdullokh3/AI_Application_final_exam/blob/main/week_14/Week_14_Lab_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a companion notebook for the book Deep Learning with Python, Second Edition. For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**
 
## **Beyond text classification: Sequence-to-sequence learning**

 **A machine translation example**

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip


--2022-12-08 14:39:31--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 142.250.4.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  2.23MB/s    in 1.1s    

2022-12-08 14:39:32 (2.23 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))



In week 14 Lab part 2 we will learn the Sequence-to-sequence learning. 

In [3]:
import random
print(random.choice(text_pairs))


("Halley's Comet will come back in 2061.", '[start] El cometa Halley volverá el 2061. [end]')


In [4]:

import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]


**Vectorizing the English and Spanish text pairs**

In [5]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)


**Preparing datasets for the translation task**

In [6]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")


inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


## **Sequence-to-sequence learning with RNNs**

**GRU-based encoder**

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)


**GRU-based decoder and the end-to-end model**

In [9]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)


**Training our recurrent sequence-to-sequence model**

In [10]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)


Epoch 1/15
1302/1302 [==============================] - 136s 92ms/step - loss: 1.6340 - accuracy: 0.4181 - val_loss: 1.3235 - val_accuracy: 0.5018
Epoch 2/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.3135 - accuracy: 0.5275 - val_loss: 1.1650 - val_accuracy: 0.5657
Epoch 3/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.1717 - accuracy: 0.5769 - val_loss: 1.0818 - val_accuracy: 0.5975
Epoch 4/15
1302/1302 [==============================] - 117s 90ms/step - loss: 1.0806 - accuracy: 0.6092 - val_loss: 1.0488 - val_accuracy: 0.6149
Epoch 5/15
1302/1302 [==============================] - 116s 89ms/step - loss: 1.0345 - accuracy: 0.6333 - val_loss: 1.0303 - val_accuracy: 0.6272
Epoch 6/15
1302/1302 [==============================] - 117s 90ms/step - loss: 1.0034 - accuracy: 0.6516 - val_loss: 1.0290 - val_accuracy: 0.6315
Epoch 7/15
1302/1302 [==============================] - 117s 90ms/step - loss: 0.9836 - accuracy: 0.6651 - val_loss: 1

**Translating new sentences with our RNN encoder and decoder**

In [11]:

import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


     

-
You can invite other people.
1/1 [==============================] - 0s 18ms/step
[start] puedes puedes personas [end]
-
You are prohibited from smoking here.
1/1 [==============================] - 0s 20ms/step
[start] tú estás de fumar aquí [end]
-
Tom had some money stolen from his tent.
1/1 [==============================] - 0s 25ms/step
[start] tom tenía algo de dinero a su de no [end]
-
He was exhausted when he got home.
1/1 [==============================] - 0s 22ms/step
[start] Él estaba [UNK] cuando se casa [end]
-
He doesn't show his true feelings.
1/1 [==============================] - 0s 19ms/step
[start] Él no le [UNK] sus sus errores [end]
-
Accidents happen.
1/1 [==============================] - 0s 18ms/step
[start] los [UNK] [UNK] [end]
-
I have ten times as many books as you have.
1/1 [==============================] - 0s 22ms/step
[start] tengo cuatro años más grande que tú [end]
-
He is what is called a self-made man.
1/1 [==============================] - 0s 28ms/s

# **Sequence-to-sequence learning with Transformer**

**The Transformer decoder**

In [12]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


Sequence-to-sequence learning with RNNs, creating GRU-based encoder GRU-based decoder and the end-to-end model, we trained our recurrent sequence-to-sequence model. And then, Translating new sentences with our RNN encoder and decoder

# **Putting it all together: A Transformer for machine translation**

**PositionalEmbedding layer**

In [13]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [14]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


In [15]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


**Training the sequence-to-sequence Transformer**

In [16]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
1302/1302 [==============================] - 98s 72ms/step - loss: 1.6411 - accuracy: 0.4312 - val_loss: 1.3243 - val_accuracy: 0.5159
Epoch 2/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.3363 - accuracy: 0.5331 - val_loss: 1.1810 - val_accuracy: 0.5716
Epoch 3/30
1302/1302 [==============================] - 93s 72ms/step - loss: 1.1981 - accuracy: 0.5762 - val_loss: 1.0960 - val_accuracy: 0.5987
Epoch 4/30
1302/1302 [==============================] - 93s 72ms/step - loss: 1.1133 - accuracy: 0.6060 - val_loss: 1.0511 - val_accuracy: 0.6178
Epoch 5/30
1302/1302 [==============================] - 93s 72ms/step - loss: 1.0655 - accuracy: 0.6273 - val_loss: 1.0328 - val_accuracy: 0.6263
Epoch 6/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.0348 - accuracy: 0.6432 - val_loss: 1.0262 - val_accuracy: 0.6338
Epoch 7/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.0131 - accuracy: 0.6561 - val_loss: 1.0092 -

**Translating new sentences with our Transformer model**

In [17]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


-
I prefer spring to summer.
[start] preferiría el año de primavera [end]
-
I didn't feel like scolding her for being late.
[start] no tenía ganas de que [UNK] su una tarea [end]
-
I'm not as young as I used to be.
[start] no soy tan joven como solía ser [end]
-
I looked for the key under the welcome mat and in the nearby flower pot.
[start] yo me quedé esperando su llave en la [UNK] y en la habitación en la habitación en la [UNK] [end]
-
Who gave you this?
[start] quién te ha dicho esto [end]
-
Give me a hundred dollars.
[start] dame una [UNK] [end]
-
We do all kinds of repairs.
[start] todos hemos tenido clase de la que [UNK] [end]
-
He's a friend from work.
[start] Él es un amigo trabajo [end]
-
Tom's sister Mary is now in Boston.
[start] la hermana de tom ahora está en boston [end]
-
I do not feel like drinking beer tonight.
[start] no tengo ganas de beber esta noche [end]
-
Things are fine.
[start] las cosas están [UNK] [end]
-
I need to get to work.
[start] necesito hacer el trab

Sequence-to-sequence learning with Transformer First, we defined The TransformerDecoder and The TransformerEncoder and Putting it all together: A Transformer for machine translation, we trained the sequence-to-sequence Transformer, Finally, translated new sentences with our Transformer mode

**Summary**